#### 사용자 정의 데이터셋과 모델과 학습
- iris.csv ==> 사용자정의 데이터셋
- DNN 모델 ==> 사용자 정의 모델

[1] 모듈로딩 및 데이터 준비

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics.classification import F1Score
from torchinfo import summary
from torch.utils.data import Dataset,DataLoader
import pandas as pd
from sklearn.preprocessing import LabelEncoder       # 타겟칼럼 수치화

# 데이터 
DATA_FILE='../data/iris.csv'
irisDF=pd.read_csv(DATA_FILE)
irisDF

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [10]:
# 타겟 칼럼 수치화 ===> LabelEncoder
encoder=LabelEncoder()
encoder.fit(irisDF['variety'])
irisDF['variety']=encoder.transform(irisDF['variety'])

In [11]:
irisDF.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


[2] 사용자 정의 데이터셋 클레스생성

In [12]:
# 클레스목적 : 학습용 데이터셋 텐서화 및 전처리
# 클레스이름 : CustomDataSet
# 부모클레스 : torch.utils.data.Dataset
# 매개변수 : featureDF,targetDF

class CustomDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self,featureDF,targetDF):
        super().__init__()
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
    # 데이터의 개수 반환 메서드 
    def __len__(self):
        return self.n_rows
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 반환!!!
    def __getitem__(self,idx):
        featureTS=torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS,targetTS

In [13]:
# 함수기능: 파일확장자별 데이터 프레임 변환 기능
# 클레스이름 : convertDataFrame
# 매개변수 : data_path 파일 경로
# 함수 결과 : DataFrame
def convertDataFrame(data_path,exit_header=0):
    ext=data_path.rsplit('.')[-1]
    if ext=='csv':
        return pd.read_csv(data_path,header=exit_header)
    elif ext=='json':
        return pd.read_json(data_path,header=exit_header)
    elif ext in['xlsx','xls']:
        return pd.read_excel(data_path,header=exit_header)
    else:
        return pd.read_table(data_path,header=exit_header)

In [14]:
# 클레스기능: 파일기반 데이터셋
# 클레스이름 : FileDataset
# 부모클레스 : torch.utils.data.Dataset
# 매개변수 : data_path 파일 경로

class FileDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self,data_path):
        super().__init__()
        dataDF=convertDataFrame(data_path)
        self.featuresDF=dataDF[dataDF.columns[:-1]]
        self.targetDF=dataDF[dataDF.columns[-1:]]
        self.n_features=self.featuresDF.shape[1]
        self.n_rows=self.featuresDF.shape[0]

    # 데이터의 개수 반환 메서드 
    def __len__(self):
        return self.n_rows
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 반환!!!
    def __getitem__(self,idx):
        featureTS=torch.FloatTensor(self.featuresDF.iloc[idx].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS,targetTS

[3] 데이터셋 인스턴스 생성

In [15]:
# 피쳐와 타겟 추출
featureDF,targetDF=irisDF[irisDF.columns[:-1]],irisDF[irisDF.columns[-1:]]
print(f'featureDF => {featureDF.shape}, targetDF => {targetDF.shape}')

# IRIS 데이터셋 인스턴스 생성
irisDS=CustomDataset(featureDF,targetDF)


featureDF => (150, 4), targetDF => (150, 1)


In [16]:
# IRIS 데이터셋 속성
irisDS.n_features,irisDS.n_rows

(4, 150)

In [17]:
irisDS[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

[4] 데이터로더 인스턴스 생성

In [18]:
## 필요한 것 : Dataset 인스턴스, Batch_size=1 defalt
irisDL=DataLoader(irisDS,batch_size=32)

In [19]:
for dataTS,targetTS in irisDL:
    print(dataTS.shape,targetTS.shape)
    break
    

torch.Size([32, 4]) torch.Size([32, 1])


[5] 모델 준비

In [27]:
# 모델 이름 : CustomModel 
# 부모클레스 : nn.Module
# 매개변수 : NONE
# 모델 구조
# - 입력층 : 입력  4개 출력 10개  AF ReLU -> LeakyReLU
# - 은닉층 : 입력 10개 출력 30개  AF ReLU -> LeakyReLU
# - 출력층 : 입력 30개 출력  3개  AF 분류 - 다중 softmax
class CustomModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.in_layer=nn.Linear(4,10)
        self.hidden_layer=nn.Linear(10,30)
        self.out_layer=nn.Linear(30,3)
    
    # 순방향 학습 메서드
    def foward(self,x):
        y=F.relu(self.in_layer(x))
        y=F.relu(self.hidden_layer(y))
        return self.out_layer(y)

In [28]:
### 모델 인스턴스 생성
model=CustomModel()
print(model)
summary(model)

CustomModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=3, bias=True)
)


Layer (type:depth-idx)                   Param #
CustomModel                              --
├─Linear: 1-1                            50
├─Linear: 1-2                            330
├─Linear: 1-3                            93
Total params: 473
Trainable params: 473
Non-trainable params: 0

[6] 학습

In [34]:
## 배치 크기만큼 데이터와 타겟 추출해서 학습 진행
for  epoch in  range(10):
    for dataTS,targetTS in irisDL:
        # 배치크기 만큼의 학습 데이터
        print(dataTS.shape,targetTS.shape)
        # 배치크기 만큼 학습 진행
        pre_y=model(dataTS)
        print(pre_y.shape,targetTS.reshape(-1).shape)
        # 손실 계산
        loss=nn.CrossEntropyLoss()(pre_y,targetTS)

        break

torch.Size([32, 4]) torch.Size([32, 1])


NotImplementedError: Module [CustomModel] is missing the required "forward" function